<a href="https://colab.research.google.com/github/cristinarainich/DDCS/blob/main/comorbidity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import pylab as pl
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy.stats as stats

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
working_path = Path('/content/gdrive/MyDrive/KKNV/comorbidity')

In [6]:
!ls /content/gdrive/MyDrive/KKNV/comorbidity

data_1.csv  data_2.csv


In [7]:
data1 = pd.read_csv(
    working_path.joinpath('data_1.csv'),
    index_col=None
)
data2 = pd.read_csv(
    working_path.joinpath('data_2.csv'),
    index_col=None
)

In [8]:
data1.head()

,Rec_Session_Id,Block_Name,Block_Nr,Task_Name,Task_Nr,Trial_Id,Trial_Nr,01_Mutter,02_Vater,03_Großeltern,...,PI20_11,PI20_12,PI20_13,PI20_14,PI20_15,PI20_16,PI20_17,PI20_18,PI20_19,PI20_20
0,910234,block_1,1,PI20,2,1,1,NaN,NaN,NaN,...,1.0,1.0,5.0,1.0,1.0,1.0,5.0,1.0,4.0,1.0
1,910234,block_1,1,Introduction,1,1,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,910234,block_1,1,AQ,3,1,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,910234,block_1,1,ARSR-v1.1.,4,1,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,910234,block_1,1,Edinburgh Handedness Inventory,5,1,1,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def PI_test(data):
  pi_subset = data[['PI20_01', 'PI20_02', 'PI20_03', 'PI20_04', 'PI20_05',
       'PI20_06', 'PI20_07', 'PI20_08', 'PI20_09', 'PI20_10', 'PI20_11',
       'PI20_12', 'PI20_13', 'PI20_14', 'PI20_15', 'PI20_16', 'PI20_17',
       'PI20_18', 'PI20_19', 'PI20_20']]
  values = pi_subset.values[0].tolist()




In [26]:
values

[1.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 5.0,
 4.0,
 2.0,
 1.0,
 1.0,
 5.0,
 1.0,
 1.0,
 1.0,
 5.0,
 1.0,
 4.0,
 1.0]

In [10]:
data1.columns[:50]

Index(['Rec_Session_Id', 'Block_Name', 'Block_Nr', 'Task_Name', 'Task_Nr',
       'Trial_Id', 'Trial_Nr', '01_Mutter', '02_Vater', '03_Großeltern',
       '04_Schwester', '05_Bruder', '06_Kinder', '07_Nein', '08_nicht_sicher',
       'AQ_example1', 'AQ_example2', 'AQ_example3', 'AQ_example4', 'AQ_Q01',
       'AQ_Q02', 'AQ_Q03', 'AQ_Q04', 'AQ_Q05', 'AQ_Q06', 'AQ_Q07', 'AQ_Q08',
       'AQ_Q09', 'AQ_Q10', 'AQ_Q11', 'AQ_Q12', 'AQ_Q13', 'AQ_Q14', 'AQ_Q15',
       'AQ_Q16', 'AQ_Q17', 'AQ_Q18', 'AQ_Q19', 'AQ_Q20', 'AQ_Q21', 'AQ_Q22',
       'AQ_Q23', 'AQ_Q24', 'AQ_Q25', 'AQ_Q26', 'AQ_Q27', 'AQ_Q28', 'AQ_Q29',
       'AQ_Q30', 'AQ_Q31'],
      dtype='object')

In [11]:
data1.columns[50:]

Index(['AQ_Q32', 'AQ_Q33', 'AQ_Q34', 'AQ_Q35', 'AQ_Q36', 'AQ_Q37', 'AQ_Q38',
       'AQ_Q39', 'AQ_Q40', 'AQ_Q41', 'AQ_Q42', 'AQ_Q43', 'AQ_Q44', 'AQ_Q45',
       'AQ_Q46', 'AQ_Q47', 'AQ_Q48', 'AQ_Q49', 'AQ_Q50', 'ASRS_Q1', 'ASRS_Q2',
       'ASRS_Q3', 'ASRS_Q4', 'ASRS_Q5', 'ASRS_Q6', 'Condition_Id', 'GazeX',
       'GazeY', 'handedness_01', 'handedness_02', 'handedness_03',
       'handedness_04', 'PI20_01', 'PI20_02', 'PI20_03', 'PI20_04', 'PI20_05',
       'PI20_06', 'PI20_07', 'PI20_08', 'PI20_09', 'PI20_10', 'PI20_11',
       'PI20_12', 'PI20_13', 'PI20_14', 'PI20_15', 'PI20_16', 'PI20_17',
       'PI20_18', 'PI20_19', 'PI20_20'],
      dtype='object')